# Get jobs from the Buildkite API

Ensure the `BUILDKITE_PAT` environmental variable is set before running this notebook.
It can be stored in the `.env` file at the top of the repo, which is kept out of
version control.

In [ ]:
import os

if os.path.basename(os.getcwd()) == "notebooks":
    print("Moving to repo root")
    os.chdir("..")

In [ ]:
import dotenv

dotenv.load_dotenv()

In [ ]:
import os

token = os.getenv("BUILDKITE_PAT")

if not token:
    raise RuntimeError("No Buildkite token found")

In [ ]:
import requests

base_url = "https://api.buildkite.com/v2"

h = {"Authorization": f"Bearer {token}"}

In [ ]:
def get_jobs(created_from: str) -> list[dict]:
    """Get all flagship AMIP jobs since ``created_from``."""
    jobs = []
    builds = requests.get(
        base_url + "/organizations/clima/pipelines/climacoupler-coarse-nightly-amip/builds",
        headers=h,
        params={"created_from": created_from},
    ).json()
    for b in builds:
        for j in b["jobs"]:
            name = j.get("name") or ""
            if "flagship amip" in name.lower() and "16 helem" in name.lower():
                job = {
                    "build_id": b.get("id"),
                    "build_state": b.get("state"),
                    "build_created_at": b.get("created_at"),
                    "commit": b.get("commit"),
                    "branch": b.get("branch"),
                    "name": name,
                    "command": j.get("command"),
                    "state": j.get("state"),
                    "created_at": j.get("created_at"),
                }
                raw_log_url = j.get("raw_log_url")
                if raw_log_url:
                    log_resp = requests.get(raw_log_url, headers=h)
                    job["raw_log_txt"] = log_resp.text
                jobs.append(job)
    return jobs

jobs = get_jobs("2026-01-15")
print(len(jobs))
jobs[0]["created_at"]